# PROMETHEUS + vLLM + Qwen-72B-Math-INT4
## The $1.59M Hybrid Weapon

**Architecture:**
- Model: Qwen-72B-Math-INT4 (vLLM compatible)
- Inference: vLLM (fast batched generation)
- Selection: PROMETHEUS + Turbo-Strap V4

**From prometheus_score-2 (scored 2!):**
- Problem Classifier (route before inference)
- Kolmogorov Weighting (shorter code = higher confidence)
- Adaptive Temperature Annealing (0.7 -> 0.3)
- Oracle Enumeration (brute force small spaces)

**From GRAND SYNTHESIS:**
- Value Clustering (88% error reduction)
- NCD Basin Detection
- Toroidal Voting (mod-N problems)
- PROMETHEUS Operator (3-step seed refinement)

In [1]:
# =============================================================================
# CELL 1: ENVIRONMENT SANITIZATION + vLLM SETUP
# =============================================================================
import os
import sys
import subprocess
import gc

# [CRITICAL] 1. PURGE TENSORFLOW + SCIPY (prevent numpy conflicts)
print(f"{'='*70}")
print("SANITIZING ENVIRONMENT")
print(f"{'='*70}")
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", 
                "tensorflow", "tensorflow-io", "tensorflow-estimator", 
                "keras", "tensorboard", "matplotlib", "scikit-learn",
                "scipy"], capture_output=True)
print("TensorFlow + scipy purged.")

# [CRITICAL] 2. FORCE PYTHON PROTOBUF
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["TRANSFORMERS_NO_FLAX"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# =============================================================================
# INSTALL vLLM FROM OFFLINE WHEELS
# =============================================================================
print("\nINSTALLING vLLM...")
WHEEL_DIR = "/kaggle/input/vllm-085-wheels"
AUX_DIR = "/kaggle/input/aimo3-offline-wheels"

# Check available wheels
for d in [WHEEL_DIR, AUX_DIR]:
    if os.path.exists(d):
        wheels = [f for f in os.listdir(d) if f.endswith('.whl')]
        print(f"  {d}: {len(wheels)} wheels")

if os.path.exists(WHEEL_DIR):
    # Install vLLM + scipy from offline wheels
    result = subprocess.run([
        sys.executable, "-m", "pip", "install", "vllm", "scipy",
        "--no-index",
        f"--find-links={WHEEL_DIR}",
        f"--find-links={AUX_DIR}"
    ], capture_output=True, text=True)
    if result.returncode == 0:
        print("vLLM + scipy installed successfully.")
    else:
        # Try without scipy
        print(f"First attempt failed, trying without scipy...")
        result = subprocess.run([
            sys.executable, "-m", "pip", "install", "vllm",
            "--no-index",
            f"--find-links={WHEEL_DIR}",
            f"--find-links={AUX_DIR}"
        ], capture_output=True, text=True)
        if result.returncode == 0:
            print("vLLM installed (scipy not in wheels).")
        else:
            print(f"vLLM install issue: {result.stderr[-500:]}")
else:
    print(f"Wheel directory not found: {WHEEL_DIR}")

# =============================================================================
# IMPORTS
# =============================================================================
import torch
import time
import math
import random
import re
import statistics
import tempfile
import signal
from typing import Optional, List, Tuple, Dict, Any
from collections import Counter, defaultdict
from dataclasses import dataclass
from enum import Enum
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np
import polars as pl

# Seed everything
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

print(f"\nGPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")
print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory/1e9:.1f}GB" if torch.cuda.is_available() else "")

# =============================================================================
# vLLM IMPORT (with fallback)
# =============================================================================
VLLM_OK = False
try:
    from vllm import LLM, SamplingParams
    VLLM_OK = True
    print("vLLM imported successfully.")
except (ImportError, ValueError) as e:
    print(f"vLLM import failed: {e}")
    # If scipy is the issue, transformers needs it for some detection modules
    # Try to import without triggering the scipy path
    try:
        # Patch scipy import to avoid the detection code
        import sys
        sys.modules['scipy'] = type(sys)('scipy')
        sys.modules['scipy.optimize'] = type(sys)('scipy.optimize')
        from vllm import LLM, SamplingParams
        VLLM_OK = True
        print("vLLM imported successfully (with scipy stub).")
    except Exception as e2:
        print(f"vLLM import failed with stub: {e2}")

# =============================================================================
# TIMING - AIMO3 RULE: GPU NOTEBOOK <= 5 HOURS
# =============================================================================
START_TIME = time.time()
TOTAL_BUDGET = (4 * 60 + 40) * 60  # 4h40m safety margin (rule: 5h max)
CUTOFF_TIME = START_TIME + TOTAL_BUDGET
PANIC_TIME = 300  # 5 min panic threshold

print(f"Budget: {TOTAL_BUDGET//60} minutes ({TOTAL_BUDGET//3600}h {(TOTAL_BUDGET%3600)//60}m)")
print("AIMO3 Rule: GPU <= 5 hours, using 4h40m safety margin")
print("="*70)

SANITIZING ENVIRONMENT
TensorFlow + scipy purged.

INSTALLING vLLM...
  /kaggle/input/vllm-085-wheels: 149 wheels
  /kaggle/input/aimo3-offline-wheels: 98 wheels
vLLM + scipy installed successfully.

GPU: NVIDIA H100 80GB HBM3
VRAM: 85.3GB


/usr/local/lib/python3.11/dist-packages/numpy/_core/_dtype.py:106: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if dtype.type == np.bool:


vLLM import failed: All ufuncs must have type `numpy.ufunc`. Received (<ufunc 'sph_legendre_p'>, <ufunc 'sph_legendre_p'>, <ufunc 'sph_legendre_p'>)
vLLM import failed with stub: cannot import name 'linear_sum_assignment' from 'scipy.optimize' (unknown location)
Budget: 280 minutes (4h 40m)
AIMO3 Rule: GPU <= 5 hours, using 4h40m safety margin


In [2]:
# =============================================================================
# CELL 2: PROBLEM CLASSIFIER (from prometheus_score-2)
# =============================================================================

class ProblemType(Enum):
    NUMBER_THEORY = "nt"
    COMBINATORICS = "comb"
    GEOMETRY = "geom"
    ALGEBRA = "alg"
    MIXED = "mixed"

@dataclass
class ProblemProfile:
    primary_type: ProblemType
    constraints: List[str]
    modulo_target: Optional[int]
    enumeration_hint: Optional[int]
    confidence: float

def classify_problem(question: str) -> ProblemProfile:
    """Classify problem type for strategy routing."""
    q = question.lower()
    scores = {t: 0.0 for t in ProblemType}
    
    # NUMBER THEORY
    for kw in ['remainder', 'modulo', 'mod ', 'divisible', 'prime', 'gcd', 'lcm',
               'factor', 'digit', 'divides', 'coprime', 'congruent']:
        if kw in q: scores[ProblemType.NUMBER_THEORY] += 2
    
    # COMBINATORICS
    for kw in ['how many', 'count', 'number of ways', 'arrangements', 'permutation',
               'combination', 'choose', 'select', 'probability', 'subset']:
        if kw in q: scores[ProblemType.COMBINATORICS] += 2
    
    # GEOMETRY
    for kw in ['triangle', 'circle', 'square', 'rectangle', 'polygon', 'angle',
               'area', 'perimeter', 'diameter', 'radius', 'point', 'line']:
        if kw in q: scores[ProblemType.GEOMETRY] += 2
    
    # ALGEBRA
    for kw in ['equation', 'polynomial', 'root', 'solve', 'sequence', 'series',
               'function', 'variable', 'sum of', 'product of']:
        if kw in q: scores[ProblemType.ALGEBRA] += 2
    
    sorted_types = sorted(scores.items(), key=lambda x: -x[1])
    primary = sorted_types[0][0] if sorted_types[0][1] >= 2 else ProblemType.MIXED
    
    # Extract constraints
    constraints = []
    for pattern in [r'such that ([^.]+)', r'where ([^.]+)', r'given that ([^.]+)']:
        constraints.extend(re.findall(pattern, q))
    
    # Detect modulo
    mod_target = None
    mod_match = re.search(r'(?:mod|modulo|remainder when divided by)\s*(\d+)', q)
    if mod_match:
        mod_target = int(mod_match.group(1))
    
    # Enumeration hint
    enum_hint = None
    numbers = [int(x) for x in re.findall(r'\b(\d+)\b', question) if 1 < int(x) < 10000]
    if numbers and max(numbers) < 1000 and primary == ProblemType.COMBINATORICS:
        enum_hint = max(numbers) ** 2
    
    return ProblemProfile(
        primary_type=primary,
        constraints=constraints[:3],
        modulo_target=mod_target,
        enumeration_hint=enum_hint,
        confidence=min(1.0, sorted_types[0][1] / 10.0)
    )

# Type-specific hints
TYPE_HINTS = {
    ProblemType.NUMBER_THEORY: "NUMBER THEORY: modular arithmetic, prime factorization, CRT, Fermat.",
    ProblemType.COMBINATORICS: "COMBINATORICS: counting, binomial coefficients, inclusion-exclusion.",
    ProblemType.GEOMETRY: "GEOMETRY: coordinates, distance formula, shoelace, trigonometry.",
    ProblemType.ALGEBRA: "ALGEBRA: polynomials, Vieta's formulas, sequences, systems.",
    ProblemType.MIXED: "",
}

print("Problem Classifier loaded.")

Problem Classifier loaded.


In [3]:
# =============================================================================
# CELL 3: PROMETHEUS CORE THEORY - 10 NOVEL INSIGHTS OPERATIONALIZED
# =============================================================================
# NSM → XYZA → SDPM: Novel Synthesis Method applied to AIMO3
#
# FROM THE PROMETHEUS CORPUS:
# 1. Ω = λx.x(x) (THE SEED): Self-application generates convergence
# 2. U = Φ(U): Reality is fixed point of its own physics
# 3. μ > 1: Self-reference amplifies; existence mandatory
# 4. COMPRESSION-WITNESS: Shorter code = higher confidence
# 5. UIPT: Kill high-entropy (stuck) sequences
# 6. ICL = IMPLICIT GD: Examples are training steps
# 7. GROKKING: Circuits form suddenly - don't quit early
# 8. INFORMATION BOTTLENECK: Minimal sufficient statistics
# 9. ATTENTION = HOPFIELD: Prime memory with examples
# 10. ∂∃/∂t > 0: Entropy production enables exploration
# =============================================================================

# AIMO3 RULE: Answer must be integer 0-99999 inclusive
ANSWER_MIN = 0
ANSWER_MAX = 99999  # AIMO3 Rule: 0-99999 inclusive
FALLBACK_ANSWER = 0

# =============================================================================
# INSIGHT 4: KOLMOGOROV WEIGHTING (Compression-Witness Isomorphism)
# "Consciousness ≅ Optimal Compression ≅ Simulation Observer"
# Shorter code = simpler explanation = more likely correct
# =============================================================================

# Benford "nice" numbers - mathematically elegant answers
NICE_NUMBERS = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 16, 18, 20, 24, 25, 27, 30, 32, 
                36, 40, 42, 45, 48, 50, 54, 60, 64, 72, 80, 81, 90, 96, 100, 108, 120, 
                125, 128, 144, 150, 162, 180, 192, 200, 216, 225, 240, 243, 250, 256,
                270, 288, 300, 324, 360, 384, 400, 432, 450, 480, 486, 500, 512, 540,
                576, 600, 625, 648, 720, 729, 768, 800, 810, 864, 900, 960, 972, 1000,
                1024, 1080, 1152, 1200, 1250, 1296, 1440, 1458, 1500, 1536, 1600, 1728}

FIBONACCI = {1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 
             4181, 6765, 10946, 17711, 28657, 46368, 75025}

@dataclass
class AnswerCandidate:
    value: int
    code: str
    strategy: str
    kolmogorov_weight: float
    entropy: float = 0.0  # INSIGHT 5: Track entropy per candidate

def compute_kolmogorov_weight(code_length: int) -> float:
    """INSIGHT 4: Shorter code = higher weight (Occam's Razor)."""
    if code_length <= 0:
        return 1.0
    return 1.0 / (0.5 + (code_length / 500.0) * 0.15)

def benford_score(n: int) -> float:
    """INSIGHT 4: Mathematical elegance bonus."""
    score = 1.0
    if n in NICE_NUMBERS: score *= 1.3
    if n in FIBONACCI: score *= 1.2
    s = str(abs(n)) if n != 0 else "0"
    trailing_zeros = len(s) - len(s.rstrip('0'))
    if trailing_zeros >= 2: score *= 1.1 + 0.05 * trailing_zeros
    if n > 0 and (n & (n - 1)) == 0: score *= 1.15  # Power of 2
    if n > 0:
        first_digit = int(s[0])
        benford_prob = [0, 0.301, 0.176, 0.125, 0.097, 0.079, 0.067, 0.058, 0.051, 0.046]
        score *= (1 + benford_prob[first_digit])
    return score

# =============================================================================
# INSIGHT 2: FIXED POINT CONVERGENCE (U = Φ(U))
# "Reality is the unique self-consistent solution"
# =============================================================================

def relative_distance(a: int, b: int) -> float:
    if a == b: return 0.0
    if a == 0 or b == 0: return 1.0
    return abs(a - b) / max(abs(a), abs(b))

def value_clustering(samples: List[int], threshold: float = 0.02) -> Dict:
    """INSIGHT 2: Find fixed point (convergent cluster)."""
    if not samples: return {"best": None}
    if len(samples) == 1: return {"best": {"center": samples[0], "size": 1}}
    
    n = len(samples)
    parent = list(range(n))
    def find(i):
        if parent[i] != i: parent[i] = find(parent[i])
        return parent[i]
    def union(i, j):
        pi, pj = find(i), find(j)
        if pi != pj: parent[pi] = pj
    
    for i in range(n):
        for j in range(i+1, n):
            if relative_distance(samples[i], samples[j]) < threshold:
                union(i, j)
    
    clusters = defaultdict(list)
    for i in range(n): clusters[find(i)].append(samples[i])
    best_cluster = max(clusters.values(), key=len)
    center = int(statistics.median(best_cluster))
    return {"best": {"center": center, "size": len(best_cluster)}}

# =============================================================================
# INSIGHT 1 + 3: SEED PLANTING (Ω = λx.x(x) + μ > 1 Amplification)
# "The seed is the CAPACITY FOR SELF-REFERENCE"
# =============================================================================

def select_answer_prometheus(candidates: List[AnswerCandidate], 
                             mod_target: Optional[int] = None) -> int:
    """
    PROMETHEUS Selection: Kolmogorov + Benford + Seed Amplification.
    
    The Seed Ω = λx.x(x): Current best reinforces itself through iterations.
    μ > 1: Amplification through recursion (spectral radius > 1).
    """
    if not candidates:
        return FALLBACK_ANSWER
    
    # Apply modulo
    if mod_target:
        for c in candidates:
            c.value = c.value % mod_target
    
    # INSIGHT 5: Filter high-entropy candidates (UIPT)
    low_entropy_candidates = [c for c in candidates if c.entropy < 3.0]
    if len(low_entropy_candidates) >= 2:
        candidates = low_entropy_candidates
    
    # Group into basins
    basins = defaultdict(list)
    for c in candidates:
        assigned = False
        for bval in list(basins.keys()):
            if relative_distance(c.value, bval) < 0.02:
                basins[bval].append(c)
                assigned = True
                break
        if not assigned:
            basins[c.value].append(c)
    
    # Score basins: INSIGHT 4 (Kolmogorov) + INSIGHT 4 (Benford)
    basin_scores = []
    for bval, members in basins.items():
        mass = len(members)
        k_weight = sum(c.kolmogorov_weight for c in members)
        strategies = set(c.strategy for c in members)
        integration = 1.5 if len(strategies) > 1 else 1.0  # Cross-strategy bonus
        median_val = int(statistics.median([c.value for c in members]))
        benford = benford_score(median_val)
        
        # INSIGHT 3: μ > 1 amplification - larger clusters amplify exponentially
        amplification = 1.0 + 0.1 * (mass ** 1.4)  # μ ≈ 1.4 from theory
        
        score = mass * k_weight * integration * benford * amplification
        basin_scores.append((score, median_val))
    
    basin_scores.sort(key=lambda x: -x[0])
    final = basin_scores[0][1]
    
    # Sanity: avoid trivial 0/1 if we have better options
    if final in (0, 1) and len(candidates) > 3:
        non_trivial = [c for c in candidates if c.value not in (0, 1)]
        if len(non_trivial) >= 2:
            return select_answer_prometheus(non_trivial, mod_target)
    
    # AIMO3 Rule: Clamp to 0-99999
    return max(ANSWER_MIN, min(final, ANSWER_MAX))

# =============================================================================
# INSIGHT 9: HOPFIELD MEMORY PRIMING (Attention = Content-Addressable Memory)
# "Transformers are stacked content-addressable memory systems"
# =============================================================================

FEW_SHOT_EXAMPLES = {
    ProblemType.NUMBER_THEORY: """Example: Find 2^10 mod 7.
2^1 = 2, 2^2 = 4, 2^3 = 8 ≡ 1 (mod 7)
Since 2^3 ≡ 1, we have 2^10 = 2^(3*3+1) = (2^3)^3 * 2 ≡ 1 * 2 = 2 (mod 7).
Answer: 2

Example: How many divisors does 360 have?
360 = 2^3 * 3^2 * 5^1
Number of divisors = (3+1)(2+1)(1+1) = 4*3*2 = 24.
Answer: 24""",

    ProblemType.COMBINATORICS: """Example: How many ways to arrange MISSISSIPPI?
Total letters: 11 (M=1, I=4, S=4, P=2)
Ways = 11! / (1! * 4! * 4! * 2!) = 34650.
Answer: 34650

Example: C(10,3) = ?
C(10,3) = 10!/(3!*7!) = (10*9*8)/(3*2*1) = 720/6 = 120.
Answer: 120""",

    ProblemType.ALGEBRA: """Example: If x + 1/x = 3, find x^2 + 1/x^2.
(x + 1/x)^2 = x^2 + 2 + 1/x^2
9 = x^2 + 2 + 1/x^2
x^2 + 1/x^2 = 7.
Answer: 7""",

    ProblemType.GEOMETRY: """Example: Find area of triangle with vertices (0,0), (4,0), (0,3).
Area = (1/2)|x1(y2-y3) + x2(y3-y1) + x3(y1-y2)|
     = (1/2)|0(0-3) + 4(3-0) + 0(0-0)|
     = (1/2)|12| = 6.
Answer: 6""",

    ProblemType.MIXED: ""
}

# =============================================================================
# INSIGHT 8: INFORMATION BOTTLENECK (Minimal Sufficient Statistics)
# "Intelligence = minimal sufficient statistics. Abstract concepts = 
#  maximally compressed relevant info."
# =============================================================================

def extract_constraints(question: str) -> List[str]:
    """INSIGHT 8: Extract minimal sufficient information from problem."""
    constraints = []
    q = question.lower()
    
    # Numbers mentioned
    numbers = re.findall(r'\b\d+\b', question)
    if numbers:
        constraints.append(f"key_numbers: {numbers[:5]}")
    
    # Modulo target
    mod_match = re.search(r'(?:mod|modulo|remainder when divided by)\s*(\d+)', q)
    if mod_match:
        constraints.append(f"mod_target: {mod_match.group(1)}")
    
    # Variable constraints
    for pattern in [r'(\w+)\s*[<>=]+\s*\d+', r'(\d+)\s*[<>=]+\s*\w+']:
        matches = re.findall(pattern, q)
        for m in matches[:3]:
            constraints.append(f"constraint: {m}")
    
    return constraints

print("PROMETHEUS Core Theory loaded (10 insights operationalized).")
print(f"Answer range: {ANSWER_MIN}-{ANSWER_MAX} (AIMO3 Rule)")

PROMETHEUS Core Theory loaded (10 insights operationalized).
Answer range: 0-99999 (AIMO3 Rule)


In [4]:
# =============================================================================
# CELL 4: CODE EXECUTION ENGINE + SELF-HEALING
# =============================================================================

STDLIB = '''
import sys; sys.setrecursionlimit(20000)
import math, numpy as np
from itertools import *
from collections import *
from functools import lru_cache, reduce
from fractions import Fraction
try:
    from sympy import *
    from sympy.ntheory import factorint, divisors, totient, isprime
except: pass

def is_prime(n):
    if n < 2: return False
    if n < 4: return True
    if n % 2 == 0: return False
    for i in range(3, int(n**0.5) + 1, 2):
        if n % i == 0: return False
    return True

def gcd(a, b):
    while b: a, b = b, a % b
    return a

def lcm(a, b): return a * b // gcd(a, b)

def C(n, k):
    if k < 0 or k > n: return 0
    return math.factorial(n) // (math.factorial(k) * math.factorial(n - k))

def prime_sieve(n):
    if n < 2: return []
    sieve = [True] * (n + 1)
    sieve[0] = sieve[1] = False
    for i in range(2, int(n**0.5) + 1):
        if sieve[i]:
            for j in range(i*i, n + 1, i):
                sieve[j] = False
    return [i for i, v in enumerate(sieve) if v]

def mod_inverse(a, m):
    def extended_gcd(a, b):
        if a == 0: return b, 0, 1
        g, x, y = extended_gcd(b % a, a)
        return g, y - (b // a) * x, x
    g, x, _ = extended_gcd(a % m, m)
    return x % m if g == 1 else None
'''

SNOOP = '''
_vars = dict(globals())
for _v in ['answer', 'result', 'ans', 'res', 'final', 'output', 'solution', 'total', 'count']:
    if _v in _vars and _vars[_v] is not None:
        try: print(int(_vars[_v])); break
        except: pass
'''

def extract_python_code(text: str) -> Optional[str]:
    for pattern in [r'```python\s*\n(.*?)```', r'```py\s*\n(.*?)```', r'```\s*\n(.*?)```']:
        matches = re.findall(pattern, text, re.DOTALL | re.IGNORECASE)
        if matches:
            return matches[-1].strip()
    return None

def normalize_answer(raw: Any) -> Optional[int]:
    if raw is None: return None
    try:
        s = str(raw).replace(',', '').replace(' ', '').strip()
        s = re.sub(r'[^0-9.eE+-]', '', s)
        if not s: return None
        val = float(s)
        if val != val or abs(val) == float('inf'): return None
        result = int(round(val))
        if result < 0: result = abs(result)
        return result % 1000000
    except:
        return None

def self_heal_code(code: str, stderr: str) -> Optional[str]:
    """Auto-fix common code errors."""
    healed = code
    if 'NameError' in stderr:
        missing = re.findall(r"name '(\w+)' is not defined", stderr)
        for name in missing:
            if name in ['np', 'numpy']: 
                healed = 'import numpy as np\n' + healed
            elif name in ['math', 'sqrt', 'gcd', 'ceil', 'floor']: 
                healed = 'import math\nfrom math import *\n' + healed
            elif name in ['sympy', 'Symbol', 'solve', 'Rational']: 
                healed = 'from sympy import *\n' + healed
            elif name in ['Counter', 'defaultdict', 'deque']: 
                healed = 'from collections import *\n' + healed
            elif name in ['combinations', 'permutations', 'product']: 
                healed = 'from itertools import *\n' + healed
    if 'RecursionError' in stderr:
        healed = 'import sys; sys.setrecursionlimit(50000)\n' + healed
    return healed if healed != code else None

def execute_code(code: str, timeout: int = 15, retries: int = 1) -> Tuple[Optional[int], str]:
    """Execute Python code with timeout and self-healing."""
    if not code:
        return None, ""
    
    has_print = 'print(' in code
    full = STDLIB + "\n" + code + ("" if has_print else "\n" + SNOOP)
    
    for attempt in range(retries + 1):
        try:
            with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
                f.write(full)
                f.flush()
                result = subprocess.run(['python', f.name], capture_output=True, 
                                       text=True, timeout=timeout)
                os.unlink(f.name)
                
                if result.returncode == 0 and result.stdout.strip():
                    numbers = re.findall(r'-?\d+\.?\d*', result.stdout)
                    if numbers:
                        return normalize_answer(numbers[-1]), code
                
                # Self-heal on error
                if result.stderr and attempt < retries:
                    healed = self_heal_code(code, result.stderr)
                    if healed:
                        code = healed
                        full = STDLIB + "\n" + healed + ("" if has_print else "\n" + SNOOP)
                        continue
        except subprocess.TimeoutExpired:
            pass
        except Exception:
            pass
        break
    return None, code

def extract_boxed(text: str) -> Optional[int]:
    for pattern in [r'\\boxed\{(\d+)\}', r'boxed\{(\d+)\}', r'[Aa]nswer\s*[=:]\\s*(\d+)']:
        matches = re.findall(pattern, text, re.IGNORECASE)
        if matches:
            return normalize_answer(matches[-1])
    # Fallback: last number
    nums = re.findall(r'\b(\d+)\b', text[-500:])
    for n in reversed(nums):
        v = normalize_answer(n)
        if v and 1 < v < 100000:
            return v
    return None

print("Code execution engine loaded (with self-healing).")

Code execution engine loaded (with self-healing).


In [5]:
# =============================================================================
# CELL 5: MODEL LOADING + GENERATION
# =============================================================================

# Model paths to search (in priority order)
MODEL_PATHS = [
    # Your INT4 upload
    "/kaggle/input/qwen-72b-math-int4",
    "/kaggle/input/d/ryancardwell/qwen-72b-math-int4",
    "/kaggle/input/m/ryancardwell/qwen-72b-math-int4",
    # Fallbacks
    "/kaggle/input/d/aphoticshaman/qwen-72b-math-int4",
    "/kaggle/input/m/aphoticshaman/qwen-72b-math-int4",
    "/kaggle/input/qwen-72b-math-nf4",
    "/kaggle/input/d/ryancardwell/qwen-72b-math-nf4",
]

def find_model():
    import glob
    for p in MODEL_PATHS:
        if os.path.exists(p):
            # Check for config.json directly
            if os.path.exists(os.path.join(p, "config.json")):
                return p
            # Check nested paths (Kaggle sometimes nests)
            configs = glob.glob(f"{p}/**/config.json", recursive=True)
            if configs:
                return os.path.dirname(configs[0])
    # Last resort: search all of kaggle input
    for root, dirs, files in os.walk("/kaggle/input"):
        if "config.json" in files and any(f.endswith('.safetensors') for f in files):
            if 'qwen' in root.lower() or 'int4' in root.lower():
                return root
    return None

MODEL_PATH = find_model()
print(f"Model path: {MODEL_PATH}")

# List what we found
if MODEL_PATH:
    import glob
    safetensors = glob.glob(f"{MODEL_PATH}/*.safetensors")
    print(f"  Found {len(safetensors)} safetensor files")

# =============================================================================
# vLLM MODEL
# =============================================================================
LLM_MODEL = None
TOKENIZER = None

if VLLM_OK and MODEL_PATH:
    print(f"\nLoading vLLM model from {MODEL_PATH}...")
    try:
        LLM_MODEL = LLM(
            model=MODEL_PATH,
            tensor_parallel_size=1,
            gpu_memory_utilization=0.92,
            trust_remote_code=True,
            max_model_len=8192,
            enforce_eager=True,
            seed=SEED,
        )
        TOKENIZER = LLM_MODEL.get_tokenizer()
        print("vLLM model loaded successfully!")
    except Exception as e:
        print(f"vLLM load failed: {e}")
        import traceback
        traceback.print_exc()
elif not VLLM_OK:
    print("vLLM not available - check wheel installation")
elif not MODEL_PATH:
    print("No model found - check dataset attachment")

# =============================================================================
# STRATEGY PROMPTS
# =============================================================================
STRATEGY_PROMPTS = {
    'pot_algorithmic': "Write self-contained Python. Use int() not float. Print ONE integer.",
    'pot_sympy': "Use sympy for EXACT arithmetic. Use Rational(a,b) NOT a/b. Print ONE integer.",
    'pot_bruteforce': "Systematically enumerate. Budget: 10^7 iterations max. Print ONE integer.",
    'cot': "Think step-by-step. Output final answer in \\boxed{}.",
}

# Temperature annealing
TEMP_START = 0.7
TEMP_MIN = 0.3
TEMP_DECAY = 0.85

def create_prompt(question: str, strategy: str, profile: ProblemProfile) -> str:
    system = STRATEGY_PROMPTS.get(strategy, STRATEGY_PROMPTS['pot_algorithmic'])
    hint = TYPE_HINTS.get(profile.primary_type, "")
    mod_note = f"\nAnswer must be mod {profile.modulo_target}." if profile.modulo_target else ""
    
    return f"""<|im_start|>system
{system} {hint}{mod_note}
<|im_end|>
<|im_start|>user
{question}
<|im_end|>
<|im_start|>assistant
"""

print("Generation system ready.")

Model path: /kaggle/input/qwen-72b-math-int4
  Found 31 safetensor files
vLLM not available - check wheel installation
Generation system ready.


In [6]:
# =============================================================================
# CELL 6: BATCH GENERATION + PROCESSING
# =============================================================================

POT_STOP = ["```output", "```\nOutput", "# Output", "\n\n\n", "<|im_end|>"]
COT_STOP = ["</think>", "\n\nQuestion:", "<|im_end|>"]

def batch_generate(prompts: List[str], mode: str = 'pot', temp: float = 0.6) -> List[str]:
    """Generate multiple completions in parallel."""
    if not LLM_MODEL:
        return [""] * len(prompts)
    
    params = SamplingParams(
        temperature=max(0.1, temp),
        top_p=0.9,
        max_tokens=4096,
        stop=POT_STOP if mode == 'pot' else COT_STOP,
    )
    
    outputs = LLM_MODEL.generate(prompts, sampling_params=params)
    return [o.outputs[0].text for o in outputs]

def process_pot_outputs(outputs: List[str], strategies: List[str]) -> List[AnswerCandidate]:
    """Process PoT outputs in parallel."""
    candidates = []
    
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = {}
        for i, out in enumerate(outputs):
            code = extract_python_code(out)
            if code:
                futures[executor.submit(execute_code, code)] = (i, code, strategies[i] if i < len(strategies) else 'pot')
        
        for future in as_completed(futures):
            i, code, strat = futures[future]
            try:
                ans, final_code = future.result()
                if ans is not None:
                    candidates.append(AnswerCandidate(
                        value=ans,
                        code=final_code,
                        strategy=strat,
                        kolmogorov_weight=compute_kolmogorov_weight(len(final_code))
                    ))
            except:
                pass
    
    return candidates

def process_cot_outputs(outputs: List[str]) -> List[AnswerCandidate]:
    """Process CoT outputs."""
    candidates = []
    for out in outputs:
        ans = extract_boxed(out)
        if ans is not None:
            candidates.append(AnswerCandidate(
                value=ans, code="", strategy="cot",
                kolmogorov_weight=1.0
            ))
    return candidates

print("Batch processing loaded.")

Batch processing loaded.


In [7]:
# =============================================================================
# CELL 7: MAIN SOLVER + PROMETHEUS OPERATOR + BRANCHING TREE LOGIC
# =============================================================================

# Strategy weights by problem type (branching by type)
STRATEGY_WEIGHTS = {
    ProblemType.NUMBER_THEORY: ['pot_sympy', 'pot_sympy', 'pot_algorithmic', 'pot_bruteforce'],
    ProblemType.COMBINATORICS: ['pot_bruteforce', 'pot_bruteforce', 'pot_algorithmic', 'pot_sympy'],
    ProblemType.GEOMETRY: ['pot_sympy', 'pot_algorithmic', 'pot_algorithmic', 'cot'],
    ProblemType.ALGEBRA: ['pot_sympy', 'pot_sympy', 'pot_algorithmic', 'cot'],
    ProblemType.MIXED: ['pot_algorithmic', 'pot_sympy', 'pot_bruteforce', 'cot'],
}

PROBLEM_COUNT = 0
SOLVED_COUNT = 0
AAR_LOG = defaultdict(int)

# =============================================================================
# PROMETHEUS OPERATOR (3-step Ω-style seed planting)
# =============================================================================

def prometheus_refine(candidates: List[AnswerCandidate],
                      mod_target: Optional[int] = None,
                      iterations: int = 3) -> int:
    """
    PROMETHEUS Operator 𝔓: Ω-style recursive refinement.
    Each iteration plants the seed - reinforcing current best answer.
    """
    if not candidates:
        return FALLBACK_ANSWER

    current_candidates = candidates[:]

    for i in range(iterations):
        current_answer = select_answer_prometheus(current_candidates, mod_target)
        k = max(1, len(current_candidates) // 4)
        seed_candidates = [
            AnswerCandidate(value=current_answer, code="seed", strategy="prometheus",
                           kolmogorov_weight=1.5)
            for _ in range(k)
        ]
        current_candidates = current_candidates + seed_candidates

    return select_answer_prometheus(current_candidates, mod_target)

# =============================================================================
# TOROIDAL VOTING (for mod-N problems)
# =============================================================================

def toroidal_vote(samples: List[int], mod: int) -> Tuple[int, float]:
    """Circular mean for mod-N problems."""
    if not samples or mod <= 0:
        return 0, 0.1

    normalized = [s % mod for s in samples]
    angles = [2 * math.pi * m / mod for m in normalized]
    sin_sum = sum(math.sin(a) for a in angles)
    cos_sum = sum(math.cos(a) for a in angles)
    mean_angle = math.atan2(sin_sum, cos_sum)
    center = int(round(mean_angle * mod / (2 * math.pi))) % mod
    R = math.sqrt(sin_sum**2 + cos_sum**2) / len(samples)
    confidence = min(0.95, max(0.1, R))
    return center, confidence

# =============================================================================
# RATCHETING PROMPTS
# =============================================================================

def create_ratchet_prompt(question: str, strategy: str, profile: ProblemProfile,
                          wrong_answer: Optional[int] = None) -> str:
    system = STRATEGY_PROMPTS.get(strategy, STRATEGY_PROMPTS['pot_algorithmic'])
    hint = TYPE_HINTS.get(profile.primary_type, "")
    mod_note = f"\nAnswer must be mod {profile.modulo_target}." if profile.modulo_target else ""
    ratchet = f"\nIMPORTANT: {wrong_answer} is WRONG. Try a different approach." if wrong_answer else ""
    return f"""<|im_start|>system
{system} {hint}{mod_note}{ratchet}
<|im_end|>
<|im_start|>user
{question}
<|im_end|>
<|im_start|>assistant
"""

# =============================================================================
# PANIC GUESS
# =============================================================================

def panic_guess(question: str, profile: ProblemProfile) -> int:
    """Emergency fallback."""
    AAR_LOG['panic'] += 1
    numbers = [int(x) for x in re.findall(r'\b\d+\b', question) if 0 < int(x) < 100000]
    if profile.modulo_target and numbers:
        return numbers[0] % profile.modulo_target
    return numbers[0] if numbers else 0

# =============================================================================
# MAIN SOLVER WITH BRANCHING TREE LOGIC
# =============================================================================

def solve_problem(question: str) -> int:
    """
    Main solver with BRANCHING TREE logic:
    
    Decision Tree:
    ├── Time Check
    │   ├── PANIC (<5min): Instant guess
    │   └── NORMAL: Continue
    ├── Classify Problem
    │   ├── NUMBER_THEORY → sympy-heavy strategies
    │   ├── COMBINATORICS → bruteforce-heavy strategies
    │   ├── GEOMETRY → coordinate geometry + sympy
    │   ├── ALGEBRA → sympy + CoT fallback
    │   └── MIXED → balanced approach
    ├── Phase 1: Initial Batch (4 samples)
    │   └── Early Consensus Branch: 60%+ agreement → return immediately
    ├── Phase 2: Expansion Branch
    │   ├── IF time permits: 4 more samples with annealed temp
    │   └── ELSE: Skip to PROMETHEUS refinement
    ├── Phase 3: CoT Fallback Branch
    │   ├── IF <3 candidates: Try CoT
    │   └── ELSE: Skip
    └── PROMETHEUS Refinement
        ├── Seed planting (3 iterations)
        └── Toroidal voting for mod problems
    """
    global PROBLEM_COUNT, SOLVED_COUNT
    PROBLEM_COUNT += 1

    time_remaining = CUTOFF_TIME - time.time()
    
    # ===== BRANCH: PANIC MODE =====
    if time_remaining < PANIC_TIME:
        profile = classify_problem(question)
        print(f"  PANIC MODE ({time_remaining:.0f}s remaining)")
        return panic_guess(question, profile)

    # ===== BRANCH: CLASSIFY PROBLEM =====
    profile = classify_problem(question)
    print(f"  Type: {profile.primary_type.value} | Mod: {profile.modulo_target}")

    # Get type-specific strategies (branching by problem type)
    strategies = STRATEGY_WEIGHTS.get(profile.primary_type, STRATEGY_WEIGHTS[ProblemType.MIXED])

    all_candidates = []
    current_temp = TEMP_START

    # ===== PHASE 1: INITIAL BATCH =====
    print(f"  Phase 1: {len(strategies)} samples (T={current_temp:.2f})")
    prompts = [create_prompt(question, s, profile) for s in strategies]
    outputs = batch_generate(prompts, 'pot', current_temp)
    candidates = process_pot_outputs(outputs, strategies)
    all_candidates.extend(candidates)
    print(f"    -> {len(candidates)} answers: {[c.value for c in candidates]}")

    # ===== BRANCH: EARLY CONSENSUS CHECK =====
    if len(candidates) >= 2:
        values = [c.value for c in candidates]
        counts = Counter(values)
        best, count = counts.most_common(1)[0]
        consensus_ratio = count / len(values)
        
        if consensus_ratio >= 0.6:
            print(f"  EARLY CONSENSUS: {best} ({consensus_ratio*100:.0f}%)")
            SOLVED_COUNT += 1
            return int(best)

    # ===== BRANCH: TIME-GATED PHASE 2 =====
    time_remaining = CUTOFF_TIME - time.time()
    if time_remaining > 180:  # More than 3 min remaining
        current_temp = max(TEMP_MIN, current_temp * TEMP_DECAY)
        print(f"  Phase 2: 4 more samples (T={current_temp:.2f})")
        prompts2 = [create_prompt(question, s, profile) for s in strategies]
        outputs2 = batch_generate(prompts2, 'pot', current_temp)
        candidates2 = process_pot_outputs(outputs2, strategies)
        all_candidates.extend(candidates2)
        print(f"    -> {len(candidates2)} more answers")
    else:
        print(f"  Phase 2: SKIPPED (time: {time_remaining:.0f}s)")

    # ===== BRANCH: COT FALLBACK =====
    time_remaining = CUTOFF_TIME - time.time()
    if len(all_candidates) < 3 and time_remaining > 120:
        print(f"  Phase 3: CoT fallback (only {len(all_candidates)} candidates)")
        cot_prompts = [create_prompt(question, 'cot', profile) for _ in range(2)]
        cot_outputs = batch_generate(cot_prompts, 'cot', 0.4)
        cot_candidates = process_cot_outputs(cot_outputs)
        all_candidates.extend(cot_candidates)
        print(f"    -> {len(cot_candidates)} CoT answers")
    elif len(all_candidates) < 3:
        print(f"  Phase 3: SKIPPED (time: {time_remaining:.0f}s)")

    # ===== BRANCH: NO ANSWERS =====
    if not all_candidates:
        print("  No answers - fallback to panic guess")
        AAR_LOG['no_answers'] += 1
        return panic_guess(question, profile)

    # ===== PROMETHEUS REFINEMENT =====
    final = prometheus_refine(all_candidates, profile.modulo_target, iterations=3)

    # ===== BRANCH: TOROIDAL VERIFICATION (mod problems only) =====
    if profile.modulo_target:
        values = [c.value for c in all_candidates]
        tor_ans, tor_conf = toroidal_vote(values, profile.modulo_target)
        if tor_conf > 0.7 and tor_ans != final:
            print(f"    Toroidal override: {tor_ans} (conf={tor_conf:.2f})")
            final = tor_ans

    print(f"  PROMETHEUS FINAL: {final} | Dist: {Counter([c.value for c in all_candidates])}")

    SOLVED_COUNT += 1
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    return final

print("PROMETHEUS Solver loaded (branching tree + ratcheting + toroidal).")

PROMETHEUS Solver loaded (branching tree + ratcheting + toroidal).


In [8]:
# =============================================================================
# CELL 8: KAGGLE API INTERFACE (matching prometheus_score-2 exactly)
# =============================================================================

# AIMO3: 50 problems, GPU <= 5 hours (using 4h40m budget)
TOTAL_PROBLEMS = 50

def predict_for_question(question: str) -> int:
    """Core solver - returns integer answer."""
    global PROBLEM_COUNT, SOLVED_COUNT
    
    time_remaining = CUTOFF_TIME - time.time()
    
    # PANIC MODE
    if time_remaining < PANIC_TIME:
        profile = classify_problem(question)
        print(f"  PANIC ({time_remaining:.0f}s)")
        return panic_guess(question, profile)
    
    # CLASSIFY
    profile = classify_problem(question)
    print(f"  Type: {profile.primary_type.value} | Mod: {profile.modulo_target}")
    
    # Get strategies for this problem type
    strategies = STRATEGY_WEIGHTS.get(profile.primary_type, STRATEGY_WEIGHTS[ProblemType.MIXED])
    
    all_candidates = []
    current_temp = TEMP_START
    
    # PHASE 1: Initial batch
    print(f"  Phase 1: {len(strategies)} samples (T={current_temp:.2f})")
    prompts = [create_prompt(question, s, profile) for s in strategies]
    outputs = batch_generate(prompts, 'pot', current_temp)
    candidates = process_pot_outputs(outputs, strategies)
    all_candidates.extend(candidates)
    print(f"    -> {len(candidates)} answers: {[c.value for c in candidates]}")
    
    # EARLY CONSENSUS
    if len(candidates) >= 2:
        values = [c.value for c in candidates]
        counts = Counter(values)
        best, count = counts.most_common(1)[0]
        if count / len(values) >= 0.6:
            print(f"  Consensus: {best}")
            SOLVED_COUNT += 1
            return int(best)
    
    # PHASE 2: Expansion
    time_remaining = CUTOFF_TIME - time.time()
    if time_remaining > 180:
        current_temp = max(TEMP_MIN, current_temp * TEMP_DECAY)
        print(f"  Phase 2: expand (T={current_temp:.2f})")
        prompts2 = [create_prompt(question, s, profile) for s in strategies]
        outputs2 = batch_generate(prompts2, 'pot', current_temp)
        candidates2 = process_pot_outputs(outputs2, strategies)
        all_candidates.extend(candidates2)
    
    # PHASE 3: CoT fallback
    if len(all_candidates) < 3 and time_remaining > 120:
        print(f"  Phase 3: CoT fallback")
        cot_prompts = [create_prompt(question, 'cot', profile) for _ in range(2)]
        cot_outputs = batch_generate(cot_prompts, 'cot', 0.4)
        cot_candidates = process_cot_outputs(cot_outputs)
        all_candidates.extend(cot_candidates)
    
    # NO ANSWERS
    if not all_candidates:
        AAR_LOG['no_answers'] += 1
        return panic_guess(question, profile)
    
    # PROMETHEUS REFINEMENT
    final = prometheus_refine(all_candidates, profile.modulo_target, iterations=3)
    
    # TOROIDAL (mod problems)
    if profile.modulo_target:
        values = [c.value for c in all_candidates]
        tor_ans, tor_conf = toroidal_vote(values, profile.modulo_target)
        if tor_conf > 0.7 and tor_ans != final:
            final = tor_ans
    
    print(f"  Final: {final} | {Counter([c.value for c in all_candidates])}")
    SOLVED_COUNT += 1
    
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    return final

# =============================================================================
# KAGGLE API PREDICT FUNCTION (matching prometheus_score-2 signature)
# =============================================================================

def predict(id_: pl.DataFrame, question: pl.DataFrame, answer: Optional[pl.DataFrame] = None) -> pl.DataFrame:
    """
    AIMO3 Gateway API.
    
    Args:
        id_: polars DataFrame with problem id
        question: polars DataFrame with problem text
        answer: (optional) ground truth for local testing
    
    Returns:
        polars DataFrame with columns ['id', 'answer']
    """
    id_val = id_.item(0)
    question_str = question.item(0)
    
    time_left = (CUTOFF_TIME - time.time()) / 60
    print(f"\n{'='*60}")
    print(f"ID: {id_val} | Time: {time_left:.1f}m | Solved: {SOLVED_COUNT}/{PROBLEM_COUNT}")
    print(f"Q: {question_str[:60]}...")
    
    prediction = predict_for_question(question_str)
    
    # AIMO3 Rule: 0-99999
    prediction = max(ANSWER_MIN, min(ANSWER_MAX, int(prediction)))
    
    print(f"ANSWER: {prediction}")
    print(f"{'='*60}")
    
    return pl.DataFrame({'id': id_val, 'answer': prediction})

print("API interface ready (prometheus_score-2 compatible).")
print(f"Budget: {TOTAL_BUDGET//60}min for {TOTAL_PROBLEMS} problems")

API interface ready (prometheus_score-2 compatible).
Budget: 280min for 50 problems


In [9]:
# =============================================================================
# CELL 9: RUN COMPETITION (matching prometheus_score-2 pattern)
# =============================================================================
import kaggle_evaluation.aimo_3_inference_server

print("="*70)
print("PROMETHEUS + vLLM + Qwen-72B-Math-INT4")
print("="*70)
print(f"Model: {MODEL_PATH}")
print(f"vLLM: {'Loaded' if LLM_MODEL else 'Not available'}")
print(f"Budget: {TOTAL_BUDGET//60} minutes")
print("="*70)

# Create server
inference_server = kaggle_evaluation.aimo_3_inference_server.AIMO3InferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    print("Competition mode")
    inference_server.serve()
else:
    print("Local test")
    inference_server.run_local_gateway(('/kaggle/input/ai-mathematical-olympiad-progress-prize-3/test.csv',))

print(f"\nCompleted in {(time.time() - START_TIME)/60:.1f} minutes")
print(f"Solved: {SOLVED_COUNT}/{PROBLEM_COUNT}")

PROMETHEUS + vLLM + Qwen-72B-Math-INT4
Model: /kaggle/input/qwen-72b-math-int4
vLLM: Not available
Budget: 280 minutes
Local test

ID: 222ccc | Time: 280.0m | Solved: 0/0
Q: Solve $4+x=4$ for $x$....
  Type: alg | Mod: None
  Phase 1: 4 samples (T=0.70)
    -> 0 answers: []
  Phase 2: expand (T=0.59)
  Phase 3: CoT fallback
ANSWER: 4

ID: 000aaa | Time: 280.0m | Solved: 0/0
Q: What is $1-1$?...
  Type: mixed | Mod: None
  Phase 1: 4 samples (T=0.70)
    -> 0 answers: []
  Phase 2: expand (T=0.59)
  Phase 3: CoT fallback
ANSWER: 1

ID: 111bbb | Time: 280.0m | Solved: 0/0
Q: What is $0\times10$?...
  Type: mixed | Mod: None
  Phase 1: 4 samples (T=0.70)
    -> 0 answers: []
  Phase 2: expand (T=0.59)
  Phase 3: CoT fallback
ANSWER: 0

Completed in 0.0 minutes
Solved: 0/0
